In [1]:
import warnings; warnings.filterwarnings("ignore")

In [2]:
import os, sys, json
import numpy as np
import pandas as pd
import seaborn as sns

from copy import copy
from glob import glob
from tqdm.auto import tqdm as tqdm
import matplotlib.pyplot as plt

In [3]:
from toolbox.reliability import split_half
from scipy.stats import pearsonr, spearmanr

In [4]:
from scipy import stats
import math

def calculate_ci(sample, confidence = 0.95):
    n, stdev = sample.shape[0], sample.std()
    test_stat = stats.norm.ppf((confidence + 1)/2)
    standard_error = test_stat * stdev / math.sqrt(n)
    return standard_error

def split_half(x, n_splits=100, mode='spearman-brown', use_tqdm = True,
               confidence_intervals = 'normal'):
    
    """Computes the split-half reliability, which speaks to the internal
    consistency of the measurement.
    
    Arguments
    
    x           -   A NumPy array with shape (M,N), where M is the number of
                    observations and N is the number of participants or tests.
                    M will be split in half to compute the reliability, not N!
    
    Keyword Arguments
    
    n_splits    -   An integer that indicates the number of times you would
                    like to split the data in X. Default value is 100.
    
    mode        -   A string that indicates the type of split-half reliability.
                    You can choose from: 'correlate' or 'spearman-brown'.
                    Default value is 'spearman-brown'.
    
    Returns
    (r, sem)    -   r is the average split-half reliability over n_splits.
                    sem standard error of the mean split-half reliability.
    """
    
    # Check the input.
    if n_splits < 1:
        raise Exception("Expected n_splits to be 1 or more, not '%s'." % \
            (n_splits))
    allowed_modes = ['correlation', 'spearman-brown']
    if mode not in allowed_modes:
        raise Exception("Mode '%s' not supported! Please use a mode from %s" \
            % (mode, allowed_modes))
    
    # Get the number of observations per subject, and the number of subjects.
    n_observations, n_subjects = x.shape
    
    # Compute the size of each group.
    n_half_1 = n_observations//2
    n_half_2 = n_observations - n_half_1
    # Generate a split-half-able vector. Assign the first half 1 and the
    # second half 2.
    halves = np.ones((n_observations, n_subjects), dtype=int)
    halves[n_half_1:, :] = 2
    
    # Run through all runs.
    r_ = np.zeros(n_splits, dtype=float)
    iterator = tqdm(range(n_splits), leave = False) if use_tqdm else range(n_splits)
    for i in iterator:

        # Shuffle the split-half vector along the first axis.
        np.random.shuffle(halves)

        # Split the data into two groups.
        x_1 = np.reshape(x[halves==1], (n_half_1, n_subjects))
        x_2 = np.reshape(x[halves==2], (n_half_2, n_subjects))
        
        # Compute the averages for each group.
        m_1 = np.nanmean(x_1, axis=0)
        m_2 = np.nanmean(x_2, axis=0)
        
        # Compute the correlation between the two averages.
        pearson_r, p = pearsonr(m_1, m_2)

        # Store the correlation coefficient.
        if mode == 'correlation':
            r_[i] = pearson_r
        elif mode == 'spearman-brown':
            r_[i] = 2.0 * pearson_r / (1.0 + pearson_r)
    
    # Compute the average R value.
    r = np.nanmean(r_, axis=0)
    # Compute the standard error of the mean of R.
    sem = np.nanstd(r_, axis=0) / np.sqrt(n_splits)
    
    confidence_intervals = 'quantile'
    if confidence_intervals == 'normal':
        ci = calculate_ci(r_, confidence = 0.95)
        ci_lower = r - ci
        ci_upper = r + ci
    if confidence_intervals == 'quantile':
        ci_lower = np.quantile(r_, 0.025)
        ci_upper = np.quantile(r_, 0.975)
    
    return r, sem, ci_lower, ci_upper

### Vessel Data Response Statistics

In [5]:
vessel_subject_data = (pd.read_csv('response/vessel_subject_data.csv')
                 .groupby(['Subj','ImageType','Image'])
                 .agg({'Rating': 'mean', 'RT': 'mean'}).reset_index())
vessel_subject_data.columns = ['subject','image_type','image_name','rating','reaction_time']

In [6]:
oracle_corr_dictlist = []
data_i = copy(vessel_subject_data)
for image_type in data_i['image_type'].unique().tolist() + ['Combo']:
    if image_type != 'Combo':
        data_i_subset = data_i[data_i['image_type'] == image_type]
    if image_type == 'Combo':
        data_i_subset = data_i
    for subject in data_i_subset['subject'].unique():
        subject_data_i = data_i_subset[data_i_subset['subject'] == subject]
        subject_item_count = len(subject_data_i['image_name'].unique())
        subject_ratings_subset = subject_data_i['image_type'].unique()
        if not ((image_type == 'Combo') & (len(subject_ratings_subset) < 5)):
            group_data_i = (data_i_subset[(data_i_subset['subject'] != subject) &
                                          (data_i_subset['image_type'].isin(subject_ratings_subset))]
                            .groupby('image_name')['rating'].mean().reset_index()
                            .sort_values(by='image_name')['rating']).to_numpy()
            subject_data_i = (data_i_subset[data_i_subset['subject'] == subject]
                              .sort_values(by='image_name')['rating']).to_numpy()
            oracle_corr_dictlist.append({'subject': subject, 'image_type': image_type, 'item_count': subject_item_count,
                                         'oracle_corr': pearsonr(subject_data_i, group_data_i)[0]})

vessel_oracle_data = pd.DataFrame(oracle_corr_dictlist)

In [22]:
splithalf_corr_dictlist = []

data_i = copy(vessel_subject_data).drop('reaction_time', 1)
for image_type in tqdm(data_i['image_type'].unique().tolist() + ['Combo']):
    if image_type != 'Combo':
        data_i_subset = data_i[data_i['image_type'] == image_type]
    if image_type == 'Combo':
        data_i_subset = data_i
    data_i_subset = data_i_subset.pivot(index='subject', columns='image_name', values='rating').to_numpy()
    splithalf_i = split_half(data_i_subset, n_splits=1000)
    splithalf_corr_dictlist.append({'measurement': 'beauty', 'image_type': image_type,  'splithalf_r': splithalf_i[0],
                                    'splithalf_lower': splithalf_i[2], 'splithalf_upper': splithalf_i[3]})

vessel_splithalf_data = pd.DataFrame(splithalf_corr_dictlist)

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [23]:
vessel_splithalf_data

,measurement,image_type,splithalf_r,splithalf_lower,splithalf_upper
0,beauty,are,0.780582,0.615302,0.874709
1,beauty,ari,0.794794,0.634008,0.879179
2,beauty,art,0.715290,0.544441,0.826396
3,beauty,fac,0.980526,0.966310,0.988413
4,beauty,lsc,0.911915,0.867741,0.940902
5,beauty,Combo,0.862447,0.813922,0.896929


In [9]:
vessel_oracle_data.to_csv('response/vessel_oracle_data.csv', index = None)

### Oasis Data Response Statistics

In [10]:
oasis_subject_data = pd.read_csv('response/oasis_subject_data.csv')

In [11]:
oracle_corr_dictlist = []
data_i = copy(oasis_subject_data)
for measurement in tqdm(['arousal','valence']):
    data_i_sub1= data_i[['subject', 'item', 'category', measurement]]
    for category in tqdm(data_i['category'].unique().tolist() + ['Combo'], leave = False):
        if category != 'Combo':
            data_i_subset = data_i_sub1[data_i_sub1['category'] == category]
        if category == 'Combo':
            data_i_subset = data_i_sub1
        for subject in tqdm(data_i_subset['subject'].unique(), leave = False):
            group_data_i = (data_i_subset[data_i_subset['subject'] != subject].groupby('item')[measurement]
                            .mean().reset_index()[measurement]).to_numpy()
            subject_data_i = data_i_subset[data_i_subset['subject'] == subject][measurement].to_numpy()
            item_indices = np.argwhere(~np.isnan(subject_data_i)).flatten()
            if len(item_indices) > 10:
                x, y = group_data_i[item_indices], subject_data_i[item_indices]
                oracle_corr_dictlist.append({'subject': subject, 'measurement': measurement, 'category': category, 
                                             'item_count': len(item_indices), 'oracle_corr': pearsonr(x, y)[0]})
                
oasis_oracle_data = pd.DataFrame(oracle_corr_dictlist)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/822 [00:00<?, ?it/s]

  0%|          | 0/822 [00:00<?, ?it/s]

  0%|          | 0/822 [00:00<?, ?it/s]

...

  0%|          | 0/822 [00:00<?, ?it/s]

  0%|          | 0/822 [00:00<?, ?it/s]

  0%|          | 0/822 [00:00<?, ?it/s]

  0%|          | 0/822 [00:00<?, ?it/s]

  0%|          | 0/822 [00:00<?, ?it/s]

In [12]:
oasis_oracle_data.groupby(['measurement','category'])['oracle_corr'].mean().reset_index()

,measurement,category,oracle_corr
0,arousal,Animal,0.339550
1,arousal,Combo,0.481066
2,arousal,Object,0.487534
3,arousal,Person,0.421060
4,arousal,Scene,0.438766
5,valence,Animal,0.742245
6,valence,Combo,0.752268
7,valence,Object,0.723934
8,valence,Person,0.739536
9,valence,Scene,0.807237


In [13]:
splithalf_corr_dictlist = []

data_i = copy(oasis_subject_data)
for measurement in tqdm(['arousal','valence']):
    data_i_sub1 = data_i[['subject', 'item', 'category', measurement]]
    for category in tqdm(data_i['category'].unique().tolist() + ['Combo'], leave = False):
        if category != 'Combo':
            data_i_subset = data_i_sub1[data_i_sub1['category'] == category]
        if category == 'Combo':
            data_i_subset = data_i_sub1
        data_i_subset = data_i_subset.pivot(index='subject', columns='item', values=measurement).to_numpy()
        splithalf_i = split_half(data_i_subset, n_splits=1000)
        splithalf_corr_dictlist.append({'measurement': measurement, 'category': category, 
                                        'splithalf_r': splithalf_i[0],
                                        'splithalf_lower': splithalf_i[2],
                                        'splithalf_upper': splithalf_i[3]})

oasis_splithalf_data = pd.DataFrame(splithalf_corr_dictlist).rename(columns={'category':'image_type'})

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

...

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [14]:
oasis_splithalf_data

,measurement,image_type,splithalf_r,splithalf_lower,splithalf_upper
0,arousal,Object,0.965020,0.942375,0.978457
1,arousal,Person,0.942397,0.903871,0.962723
2,arousal,Animal,0.916097,0.850767,0.952001
3,arousal,Scene,0.956330,0.926580,0.973034
4,arousal,Combo,0.963349,0.945364,0.974638
5,valence,Object,0.990545,0.987110,0.993284
6,valence,Person,0.991324,0.988179,0.993627
7,valence,Animal,0.991435,0.987208,0.994151
8,valence,Scene,0.994310,0.991805,0.995900
9,valence,Combo,0.992036,0.989864,0.993671


### Aenne Data Response Statistics

In [15]:
aenne_subject_data = (pd.read_csv('response/aenne_subject_data.csv'))

In [16]:
oracle_corr_dictlist = []
data_i = copy(aenne_subject_data)
for measurement in tqdm(['beauty']):
    data_i_sub1= data_i[['subject', 'item', 'category', measurement]]
    for category in tqdm(data_i['category'].unique().tolist() + ['Combo'], leave = False):
        if category != 'Combo':
            data_i_subset = data_i_sub1[data_i_sub1['category'] == category]
        if category == 'Combo':
            data_i_subset = data_i_sub1
        for subject in tqdm(data_i_subset['subject'].unique(), leave = False):
            group_data_i = (data_i_subset[data_i_subset['subject'] != subject].groupby('item')[measurement]
                            .mean().reset_index()[measurement]).to_numpy()
            subject_data_i = data_i_subset[data_i_subset['subject'] == subject][measurement].to_numpy()
            item_indices = np.argwhere(~np.isnan(subject_data_i)).flatten()
            if len(item_indices) > 10:
                x, y = group_data_i[item_indices], subject_data_i[item_indices]
                oracle_corr_dictlist.append({'subject': subject, 'measurement': measurement, 'category': category, 
                                             'item_count': len(item_indices), 'oracle_corr': pearsonr(x, y)[0]})
                
aenne_oracle_data = pd.DataFrame(oracle_corr_dictlist)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/757 [00:00<?, ?it/s]

  0%|          | 0/757 [00:00<?, ?it/s]

  0%|          | 0/757 [00:00<?, ?it/s]

  0%|          | 0/757 [00:00<?, ?it/s]

  0%|          | 0/757 [00:00<?, ?it/s]

In [17]:
aenne_oracle_data.groupby(['measurement','category'])['oracle_corr'].mean().reset_index()

,measurement,category,oracle_corr
0,beauty,Animal,0.568543
1,beauty,Combo,0.643166
2,beauty,Object,0.609348
3,beauty,Person,0.607161
4,beauty,Scene,0.697122


In [18]:
splithalf_corr_dictlist = []

data_i = copy(aenne_subject_data)
for measurement in tqdm(['beauty']):
    data_i_sub1 = data_i[['subject', 'item', 'category', measurement]]
    for category in tqdm(data_i['category'].unique().tolist() + ['Combo']):
        if category != 'Combo':
            data_i_subset = data_i_sub1[data_i_sub1['category'] == category]
        if category == 'Combo':
            data_i_subset = data_i_sub1
        data_i_subset = data_i_subset.pivot(index='subject', columns='item', values=measurement).to_numpy()
        splithalf_i = split_half(data_i_subset, n_splits=1000)
        splithalf_corr_dictlist.append({'measurement': measurement, 'category': category, 
                                        'splithalf_r': splithalf_i[0],
                                        'splithalf_lower': splithalf_i[2],
                                        'splithalf_upper': splithalf_i[3]})

aenne_splithalf_data = pd.DataFrame(splithalf_corr_dictlist).rename(columns={'category':'image_type'})

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [19]:
aenne_splithalf_data

,measurement,image_type,splithalf_r,splithalf_lower,splithalf_upper
0,beauty,Object,0.985547,0.976809,0.990471
1,beauty,Person,0.985184,0.976473,0.989972
2,beauty,Animal,0.986786,0.976600,0.992013
3,beauty,Scene,0.991782,0.987351,0.994417
4,beauty,Combo,0.988856,0.984061,0.991646


In [20]:
pd.concat([oasis_oracle_data, aenne_oracle_data]).to_csv('response/oasis_oracle_data.csv', index = None)

In [24]:
vessel_splithalf_data['dataset'] = 'vessel'
oasis_splithalf_data['dataset'] = 'oasis'
aenne_splithalf_data['dataset'] = 'oasis'
(pd.concat([vessel_splithalf_data, oasis_splithalf_data, aenne_splithalf_data])
 .to_csv('response/splithalf_data.csv', index = None))